PROBLEM STATEMENT:

The aim of the project is to process the images of kids in classroom and to identify attentive and non-attentive kids among them. 

INSTALLING AND  IMPORTING LIBRARIES:

In [1]:
pip install tensorflow 

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1.2 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [13]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator

IMAGE PREPROCESSING:

In [14]:
#configure image data generator
train_datagen=ImageDataGenerator(rescale=1./255,shear_range=0.2,zoom_range=0.2,horizontal_flip=True)
test_datagen=ImageDataGenerator(rescale=1./255)

In [15]:
#3.Apply image data generator functionality to train and test images
x_train=train_datagen.flow_from_directory(r'C:\Users\KEERTHI KRISHANA\OneDrive\Documents\train',target_size=(64,64),batch_size=32,class_mode="categorical")
x_test = test_datagen.flow_from_directory(r'C:\Users\KEERTHI KRISHANA\OneDrive\Documents\test',target_size = (64,64),batch_size = 32 , class_mode = 'categorical')

Found 8 images belonging to 2 classes.
Found 8 images belonging to 2 classes.


MODEL BUILDING:

In [16]:


# Define the number of classes (attentive or inattentive)
num_classes = 2

# Define the base model (e.g., VGG16, ResNet50, etc.)
base_model = tf.keras.applications.VGG16(weights='imagenet', include_top=False )

# Freeze the base model layers
for layer in base_model.layers:
    layer.trainable = False

# Add a new classification head
x = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)
x = tf.keras.layers.Dense(128, activation='relu')(x)
x = tf.keras.layers.Dropout(0.5)(x)
output = tf.keras.layers.Dense(num_classes, activation='softmax')(x)

# Create the final model
model = tf.keras.models.Model(inputs=base_model.input, outputs=output)

# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
              loss='categorical_crossentropy',metrics=['accuracy'])




STATISTICAL DESCRIPTION OF DATA:

In [17]:
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, None, None, 3)]   0         
                                                                 
 block1_conv1 (Conv2D)       (None, None, None, 64)    1792      
                                                                 
 block1_conv2 (Conv2D)       (None, None, None, 64)    36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, None, None, 64)    0         
                                                                 
 block2_conv1 (Conv2D)       (None, None, None, 128)   73856     
                                                                 
 block2_conv2 (Conv2D)       (None, None, None, 128)   147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, None, None, 128)   0   

TRAINING THE MODEL:

In [18]:
# Load and preprocess the data
train_data = x_train
validation_data = x_test

# Define the training parameters
batch_size = 8
epochs = 5

# Train the model
model.fit(train_data,
          epochs=epochs,
          batch_size=batch_size,
          validation_data=validation_data)


Epoch 1/5
1/1 [==============================] - 6s 6s/step - loss: 0.8497 - accuracy: 0.3750 - val_loss: 0.7115 - val_accuracy: 0.5000
Epoch 2/5
1/1 [==============================] - 1s 1s/step - loss: 0.9848 - accuracy: 0.3750 - val_loss: 0.7112 - val_accuracy: 0.5000
Epoch 3/5
1/1 [==============================] - 1s 1s/step - loss: 0.9675 - accuracy: 0.2500 - val_loss: 0.7101 - val_accuracy: 0.5000
Epoch 4/5
1/1 [==============================] - 1s 1s/step - loss: 0.9672 - accuracy: 0.1250 - val_loss: 0.7083 - val_accuracy: 0.5000
Epoch 5/5
1/1 [==============================] - 1s 1s/step - loss: 0.5282 - accuracy: 0.8750 - val_loss: 0.7065 - val_accuracy: 0.5000


SAVING THE MODEL:

In [19]:
# Save the model
model.save('mymodel.keras')


TESTING THE MODEL:

In [5]:
import cv2
import numpy as np
import tensorflow as tf

# Load pre-trained deep learning model
model = tf.keras.models.load_model('mymodel.keras')

#preprocess the image 
def preprocess_image(image):
    # Convert the image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Resize the image to a fixed size
    resized = cv2.resize(gray, (100, 100))
    
    # Convert the image to a float32 and normalize it
    normalized = resized.astype('float32') / 255.0
    
    # Add a channel dimension to the image
    preprocessed = np.expand_dims(normalized, axis=-1)
    
    return preprocessed


# Function to predict attentiveness level
def predict_attentiveness(image):
    
    # Make predictions using the deep learning model
    predictions = model.predict(np.expand_dims(image, axis=0))
    
    # Extract attentiveness level from predictions
    attentiveness_level = np.argmax(predictions)
    
    return attentiveness_level

n = int(input("enter number of images:"))
for i in range(n):
    img_path = input(f"Enter the path of image {i+1}: ")
    image = cv2.imread(img_path)
    if image is not None:
    # Predict attentiveness level
        attentiveness_level = predict_attentiveness(image)
        if(attentiveness_level == 1):
        # Display attentiveness level
            print(f"The attentiveness level of the student is: {attentiveness_level}")
            print(f"kid is attentive in class")
        else:
            
            print(f"The attentiveness level of the student is: {attentiveness_level}")
            print(f"kid is non-attentive in class")
    else:
        print("Failed to load the image. Please check the file path.")


1/1 [==============================] - 1s 714ms/step
The attentiveness level of the student is: 0
kid is non-attentive in class


CONCLUSION:

finally, we created a real time class room monitoring system to identify attentive and non-attentive kids by taking sample images of kids from the user. the built AI model will show the attentiveness level of the kid in the image as 1 if they are attentive or else it shows 0 for non-attentive kids.